In [91]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

# Linear regression is from here:
# https://www.kaggle.com/apapiu/regularized-linear-models
# I use data analysis techniques from my last python notebook

%config InlineBackend.figure_format = 'png'
%matplotlib inline

In [92]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [93]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [94]:
# the guy in the guide applies different transformations. I'll apply the ones from the other thing.
all_data = pd.concat((train.loc[:, 'MSSubClass':'SaleCondition'], test.loc[:, 'MSSubClass':'SaleCondition']))

In [95]:
# Select Variables I looked at before.
# I'm going to do a simple model with only these variables.
# In his guide, he simply log(1+x) every variabl. I'm going to apply the transformations that I did
# in my other guide and look at a simple model. I don't know if this is good or not, but
# if this simple model is accurate, I honestly prefer it over a more complex one.
vars = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt']
o_vars = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt']
all_data = all_data.loc[:,vars]
train = train.loc[:, o_vars]
# test = test.loc[:, o_vars]
# uhh why does test not have SalePrice...oh that's how kaggle works...ahhh
all_data.head()

,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,YearBuilt
0,7,1710,2.0,856.0,2003
1,6,1262,2.0,1262.0,1976
2,7,1786,2.0,920.0,2001
3,7,1717,3.0,756.0,1915
4,8,2198,3.0,1145.0,2000


In [96]:
a = all_data.isnull().sum(axis=0)
total = a.sort_values(ascending=False)
total.head(5)

# 1 Observation missing for TotalBsmtSF and GarageCars. We'll fill it cuz it's in the test set.

GarageCars     1
TotalBsmtSF    1
OverallQual    0
GrLivArea      0
YearBuilt      0
dtype: int64

In [97]:
# all_data = all_data.drop(all_data.loc[all_data['TotalBsmtSF'].isnull()].index)
# all_data = all_data.drop(all_data.loc[all_data['GarageCars'].isnull()].index)
all_data = all_data.fillna(all_data.mean())
all_data.isnull().sum().max()

0

In [98]:
# ok now that I think about it, I went about things a bit badly...really gotta plan ahead
# its ok I just want to build out the model at this moment. Let's just apply transformations.

In [99]:
# Log transform saleprice for the train data??
train['SalePrice'] = np.log(train["SalePrice"])

In [100]:
# other transforms
train['GrLivArea'] = np.log(train['GrLivArea'])

In [101]:
# last transform. I think for this one, I'll do the log1p thing from this guide. It sounds fun.
train['TotalBsmtSF'] = np.log1p(train['TotalBsmtSF'])

In [102]:
all_data.head() # uhhh

,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,YearBuilt
0,7,1710,2.0,856.0,2003
1,6,1262,2.0,1262.0,1976
2,7,1786,2.0,920.0,2001
3,7,1717,3.0,756.0,1915
4,8,2198,3.0,1145.0,2000


In [103]:
all_data = pd.get_dummies(all_data)

In [104]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
# Train.shape gives the shape of the matrix(rowsxcols), accessing the 0th element of the tuple gives us the number of rows.
# So we just kinda separated the data by rows and yeah we're done here.
y = train.SalePrice

In [105]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [106]:
regr = linear_model.LinearRegression()

In [107]:
regr.fit(X_train, y)

LinearRegression()

In [109]:
y_pred = regr.predict(X_train)

In [114]:
# coefficients
coeffs = pd.concat([pd.DataFrame(X_train.columns), pd.DataFrame(regr.coef_)], axis=1)
print("Coefficients: \n", coeffs)
print('Mean Squared Error for training set:', mean_squared_error(y, y_pred))
# mean squared error is so low...I wonder if the model is overfitted...?

Coefficients: 
              0         0
0  OverallQual  0.102888
1    GrLivArea  0.000240
2   GarageCars  0.087753
3  TotalBsmtSF  0.000124
4    YearBuilt  0.002360
Mean Squared Error for training set: 0.029994381604057305


In [77]:
test_pred = regr.predict(X_test)

In [78]:
test_pred = np.exp(test_pred)
test_pred
# Looks about right...

array([114581.76890874, 147860.37392701, 163301.2251185 , ...,
       140847.01557964, 115371.3849057 , 239159.52329116])

In [79]:
len(test_pred)
# FCK I PROBABLY REMOVED SOME ROWS OH NOOOO
# Ok we removed like 2 things total we just gotta...not remove them lol

1459

In [81]:
# SHEEEESH MOMENT OF TRUTH
# nvm we need id...
var123 = test['Id']
var123
# cool...

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [85]:
var234 = pd.concat([var123, pd.Series(test_pred)], axis=1)
var234.columns = ['Id', 'SalePrice']
var234.head()

,Id,SalePrice
0,1461,114581.768909
1,1462,147860.373927
2,1463,163301.225118
3,1464,180295.529899
4,1465,211074.910724


In [90]:
# pray this model ain't overfit :P
var234.to_csv('submission.csv', index=False)
# ok we got the set. MOMENT OF TRUTHHH

In [ ]:
# Wow...0.17155 RMSLE on the test set...
# considering I did a simple linear regression with 5 variables, not bad...
# Maybe i'll add more variables, I'll look into interaction terms?? Who knows?